In [9]:
import multiprocessing
import pycurl
import pymysql
import urllib.parse
import io
import re
import json
import requests
from lxml.html import fromstring, tostring

In [95]:
con = pymysql.connect(host='localhost', user='root', passwd='', db='xiaobaifinder', charset='utf8')
cu_r = con.cursor()
cu_r.execute('select pmid,link from doi_link where link like "%elsevier%" limit 100,10')
link_list = cu_r.fetchall()
print(link_list)
con.close()

(('26656174', 'http://linkinghub.elsevier.com/retrieve/pii/S0378111915014742'), ('26656137', 'http://linkinghub.elsevier.com/retrieve/pii/S0300908415004095'), ('26656123', 'http://linkinghub.elsevier.com/retrieve/pii/S1443950615014614'), ('26656068', 'http://linkinghub.elsevier.com/retrieve/pii/S000689931500904X'), ('26656067', 'http://linkinghub.elsevier.com/retrieve/pii/S0006899315009051'), ('26656151', 'http://linkinghub.elsevier.com/retrieve/pii/S0001457515301044'), ('26656007', 'http://linkinghub.elsevier.com/retrieve/pii/S2210740115002624'), ('26655877', 'http://linkinghub.elsevier.com/retrieve/pii/S1567576915301995'), ('26656222', 'http://linkinghub.elsevier.com/retrieve/pii/S0168165615301978'), ('26655793', 'http://linkinghub.elsevier.com/retrieve/pii/S0927776515303246'))


In [96]:
url = 'http://linkinghub.elsevier.com/retrieve/pii/S0927776515303246'

In [117]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36'}
def elsevier(pmid, url):
    response = requests.get(url)
    server = response.headers['Server']
    if server == 'www.sciencedirect.com':
        get_sciencedirect(pmid, response.content)
        
    return(response.content)

In [118]:
text = elsevier(pmid, url)

In [119]:
text

b'\n\n\n\n\n\n<!doctype html>\n<html>\n<head>\n<meta charset="utf-8"/>\n<meta http-equiv="X-UA-Compatible" content="IE=edge" />\n\n<meta name="robots" content="NOARCHIVE">\n<meta name="robots" content="noodp" />\n<meta name="robots" content="noydir" />\n\n<link rel="canonical" href="http://www.sciencedirect.com/science/article/pii/S0927776515303246">\n\n\n<link rel="shortcut icon" href="http://cdn.els-cdn.com/sd/favSD.ico" type="image/x-icon">\n<title>\n Reversing of multidrug resistance breast cancer by co-delivery of P-gp siRNA and doxorubicin via folic acid-modified core-shell nanomicelles\n </title> \n\n<link type="text/css" rel="StyleSheet" href="http://www.sciencedirect.com/gadgetservices/gadgets/v9/css/gadgets_ext,gadgets,toolbar,SDArticleGadgetOverRide,jquery-ui-1.8.4.custom.css">\n\n\n<link rel="stylesheet" href="http://cdn.els-cdn.com/sd/css/article_1601R2.css" type="text/css">\n<link rel="stylesheet" href="http://cdn.els-cdn.com/sd/css/articleSprite_1601R2.css" type="text/cs

In [131]:
def get_sciencedirect(pmid, text):
    pmid = int(pmid)
    root = fromstring(text)
#     ul = root.cssselect('ul.authorGroup')[0]
    # aff
    aff_dict = {}
    aff_index = 0
    for aff in root.cssselect('ul.affiliation li'):
        aff_index +=1
        aff_id = aff.get('id')
        aff_name = aff.find('span').text_content()
        aff_dict[aff_id] = str(aff_name)
        cu.execute('insert into journal_aff (pmid, aff_index, aff_name) values(%s, %s, %s)', (pmid, aff_index, aff_name,))
        con.commit()
    # author
    lis = [li for li in root.cssselect('ul.authorGroup li')]
    author_index = 0
    for li in lis:
        a_list = li.findall('a')
        corresp=0
        author=None
        email = None
        author_aff_name = None
        author_index +=1
        author_aff_list = []
        for a in a_list:
            if 'authorName' in a.get('class'):
                author = a.text_content()
            if a.get('title') and 'Corresponding author contact information' in a.get('title'):
                corresp = 1
                print(corresp)
            if a.get('class') and a.get('href') and 'auth_mail' in a.get('class'):
                email = a.get('href').replace('mailto:', '')
                print(email)
            if a.get('title') and 'Affiliation' in a.get('title') or len(aff_dict)==1:
                
                if len(aff_dict) == 1:
#                     author_aff_name=aff_dict[aff_id]
                    author_aff_list.append(aff_dict[aff_id])
                else:
                    aff = a.get('href').replace('#', '')
#                     author_aff_name = aff_dict[aff]
                    author_aff_list.append(aff_dict[aff])
        if author_aff_list:
            for author_aff_name in author_aff_list:
                cu.execute('insert into journal_author values(%s, %s, %s, %s, %s, %s)',
                           (pmid, author_index, author, author_aff_name, email, corresp,))
                con.commit()
        else:
            cu.execute('insert into journal_author values(%s, %s, %s, %s, %s, %s)',
                           (pmid, author_index, author, author_aff_name, email, corresp,))
            con.commit()

                

    #keywords
    keywords = root.cssselect('ul.keyword li')
    for kw in keywords:
        cu.execute('insert into journal_keyword values(%s, %s)', (pmid, kw.text_content().replace(';', '').strip(),))
        con.commit()
            

In [116]:
text

In [113]:
get_sciencedirect(1, text)

TypeError: expected string or buffer

In [132]:
con = pymysql.connect(host='localhost', user='root', passwd='', db='xiaobaifinder', charset='utf8')
cu = con.cursor()
cu.execute('create table if not exists journal_keyword(pmid int, keywords varchar(200))')
cu.execute('create table if not exists journal_aff(pmid int,aff_index int, aff_name Text)')
cu.execute('create table if not exists journal_author(pmid int,author_index int,\
            author varchar(200), author_aff Text, email varchar(200), corresp int)')
con.commit()
pmid=1
# text = elsevier(pmid, url)
get_sciencedirect(1, text)
cu.close()
con.close()






1
tangjinhai2015@sina.com


/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/pymysql/cursors.py:146: Warning: Table 'journal_keyword' already exists
  result = self._query(query)
/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/pymysql/cursors.py:146: Warning: Table 'journal_aff' already exists
  result = self._query(query)
/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/pymysql/cursors.py:146: Warning: Table 'journal_author' already exists
  result = self._query(query)


In [94]:
get_sciencedirect(text)

3
Elaine Wat
Institute of Chinese Medicine, The Chinese University of Hong Kong, Shatin, New Territories, Hong Kong SAR, PR China
State Key Laboratory of Phytochemistry and Plant Reso urces in West China, The Chinese University of Hong Kong, Shatin, New Territories, Hong Kong SAR, PR China
Chun Fai Ng
Institute of Chinese Medicine, The Chinese University of Hong Kong, Shatin, New Territories, Hong Kong SAR, PR China
State Key Laboratory of Phytochemistry and Plant Reso urces in West China, The Chinese University of Hong Kong, Shatin, New Territories, Hong Kong SAR, PR China
Eric Chun Wai Wong
Institute of Chinese Medicine, The Chinese University of Hong Kong, Shatin, New Territories, Hong Kong SAR, PR China
State Key Laboratory of Phytochemistry and Plant Reso urces in West China, The Chinese University of Hong Kong, Shatin, New Territories, Hong Kong SAR, PR China
Chi Man Koon
Institute of Chinese Medicine, The Chinese University of Hong Kong, Shatin, New Territories, Hong Kong SAR, P

In [80]:
text.headers

AttributeError: 'bytes' object has no attribute 'headers'

In [17]:
text.content

b'\n\n\n\n\n\n<!doctype html>\n<html>\n<head>\n<meta charset="utf-8"/>\n<meta http-equiv="X-UA-Compatible" content="IE=edge" />\n\n<meta name="robots" content="NOARCHIVE">\n<meta name="robots" content="noodp" />\n<meta name="robots" content="noydir" />\n\n<link rel="canonical" href="http://www.sciencedirect.com/science/article/pii/S0167488915004218">\n\n\n<link rel="shortcut icon" href="http://cdn.els-cdn.com/sd/favSD.ico" type="image/x-icon">\n<title>\n The E3 ubiquitin ligase CHIP mediates ubiquitination and proteasomal degradation of PRMT5\n </title> \n\n<link type="text/css" rel="StyleSheet" href="http://www.sciencedirect.com/gadgetservices/gadgets/v9/css/gadgets_ext,gadgets,toolbar,SDArticleGadgetOverRide,jquery-ui-1.8.4.custom.css">\n\n\n<link rel="stylesheet" href="http://cdn.els-cdn.com/sd/css/article_1601R1.css" type="text/css">\n<link rel="stylesheet" href="http://cdn.els-cdn.com/sd/css/articleSprite_1601R1.css" type="text/css">\n<link rel="stylesheet" href="http://cdn.els-cd

In [3]:
def get_content(url):
    try:
        storage = io.BytesIO()
        c = pycurl.Curl()
        c.setopt(c.URL, url)
        c.setopt(pycurl.FOLLOWLOCATION, 1) 
        c.setopt(pycurl.WRITEFUNCTION, storage.write)
        # c.setopt(c.HTTPHEADER,["Accept: application/vnd.crossref.unixsd+xml"])
        c.perform()
        out = c.getinfo(pycurl.EFFECTIVE_URL)
#         cu_w.execute('insert into doi_link values(%s, %s, %s)', (pmid,doi, out))
#         con.commit()
        c.close()
        content = storage.getvalue().decode('utf-8')
#         print(out)
#         print(content)
        storage.close()
        return content,out
    except Exception as e:
        print(e)

In [64]:
con = pymysql.connect(host='localhost', user='root', passwd='', db='xiaobaifinder', charset='utf8')
cu_r = con.cursor()
cu_r.execute('insert into tmp values(%s)', (None,))
con.commit()
cu_r.close()
con.close()